# Federated learning - Client aggregation

### Imports

In [2]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import math

In [3]:
## CONSTANTS
NUMBER_OF_CLIENTS = 10 # Currently only 10 data splits
LOCAL_EPOCHS = 3
ROUNDS = 1
LOCAL_LR = 0.01
GLOBAL_LR = 0.1


## FT CONSTANTS
FT_EPOCHS = 3

### Utility

In [4]:
def sort_lists(list_a, list_b, data_entry, data_label):
    list_a[data_label].append(data_entry)
    list_b[data_label].append(data_label)

### Prepare the data

##### MNIST

In [5]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()
train_data, train_data_labels, test_data, test_data_labels = [[[] for i in range(10)] for i in range(4)]
    
list(map(lambda a,b:sort_lists(train_data,train_data_labels, a, b), train_images, train_labels))
#list(map(lambda a,b:sort_lists(test_data,test_data_labels, a, b), test_images, test_labels))
test_data, test_data_labels = list(split(test_images, NUMBER_OF_CLIENTS)), list(split(test_labels, NUMBER_OF_CLIENTS))

train_data, train_data_labels, test_data_labels, test_data_labels = np.asarray(train_data), np.asarray(train_data_labels), np.asarray(test_data_labels), np.asarray(test_data_labels)

print(len(train_data[0]), len(train_data_labels[0]), len(test_data[0]), len(test_data_labels[0]))

NameError: name 'split' is not defined

###### CIFAR10

In [139]:
(cifar_images, cifar_labels), (cifar_test_images, cifar_test_labels) = datasets.cifar10.load_data()
cifar_train_data, cifar_train_labels, cifar_test_data, cifar_test_labels = [[[] for i in range(10)] for i in range(4)]


### Models

###### MNIST

In [140]:
mnist_model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
        ])

mnist_model.compile(
                optimizer=tf.keras.optimizers.SGD(0.01),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )

###### CIFAR10

In [141]:
# TODO

### Classes and methods

###### Server class

In [142]:
class Server():
    def __init__(self, model):
        self.clients = []
        self.client_weights = []
        self.weights = []
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
        ])
        self.model.compile(
                optimizer=tf.keras.optimizers.SGD(0.1),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )
        
    # Implementing a getter because TF get_weights returns a list rather than array.
    def get_weights(self):
        return np.asarray(self.model.get_weights())
    
    # Performs a single round of training for set amount of epochs    
    def train_clients(self):
        for c in self.clients:
            self.client_weights.append(c.train())
        return self.client_weights
    
    # Updates weights for server and clients
    def update_weights(self, new_weights):
        self.weights = new_weights
        for c in self.clients:
            c.update_weights(self.get_weights())
            
    # Performs federated averaging for given amount of rounds
    def federated_averaging(self, rnds):
        average_weights = np.asarray(self.model.get_weights())
        # Training for a specified amount of rounds
        for rnd in range(rnds):
            t_weights = []
            total_samples = 0
            # Training loop - trains each client for a single round for a specified amount of epochs
            for client in self.clients:
                t_weights.append(client.train(LOCAL_EPOCHS))

                total_samples += client.get_sample_amount()

            tmp_weights = [t_weights[i] * len(self.clients[i].dataset.testing_data) for i in range(len(self.clients))]
            tmp_weights = [w / total_samples for w in average_weights]

            average_weights = [x + y for x, y in zip(init_weights, tmp_weights)]

        self.update_weights(np.asarray(average_weights))
        print('FINISHED')

###### Client class

In [143]:
# Class representing the client and the necessary methods a client requires.
class Client():
    def __init__(self, dataset):
        self.model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
        ])
        self.model.compile(
                optimizer=tf.keras.optimizers.SGD(LOCAL_LR),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )
        self.dataset = dataset
        self.active_weights = self.get_weights()
        self.rep_weights = self.active_weights
        
    # Implementing a getter because TF get_weights returns a list rather than array.
    def get_weights(self):
        return np.asarray(self.model.get_weights())
    
    # Getter just for better readability
    def get_sample_amount(self):
        return len(self.dataset.training_data)
    
    def train(self, epochs):
        self.model.fit(
                self.dataset.training_data, 
                self.dataset.training_labels, 
                epochs=LOCAL_EPOCHS, 
                validation_data=(
                    self.dataset.testing_data,
                    self.dataset.testing_labels
                ))
        
        self.rep_weights = np.asarray(self.model.get_weights())
        return self.rep_weights
    
    def update_weights(self, new_weights):
        assert self.active_weights.shape == new_weights.shape, "Shapes for the input weights are not the same"
        self.active_weights = np.mean( np.array([ self.active_weights, new_weights ]), axis=0 )
        self.model.set_weights(self.active_weights)
        
    def fine_tune(self, new_weights):
        pretrained_model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10)
        ])
        pretrained_model.set_weights(new_weights)
        pretrained_model.trainable = True
        fine_tune_at = int(len(pretrained_model.layers) / 3)
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        pretrained_model.compile(optimizer=tf.keras.optimizers.SGD(LOCAL_LR/10),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )
        
        # train layers
        ft_history = pretrained.model.fit(
                                self.dataset.training_data,
                                self.dataset.training_labels,
                                epochs = LOCAL_EPOCHS + FT_EPOCHS,
                                validation_data=(
                                    self.dataset.testing_data,
                                    self.dataset.testing_labels
                                ))
        return ft_history
    
    def ensemble(self):
        for i, j in self.dataset.testing_data, self.dataset.testing_labels:
            # evaluate both models on each datapoint, keep both results for now.
        self.model.evaluate()

###### Dataset class

In [144]:
class Dataset():
    def __init__(self, training_data, training_labels, testing_data, testing_labels):
        self.training_data = np.asarray(training_data)
        self.training_labels = np.asarray(training_labels)
        self.testing_data = np.asarray(testing_data)
        self.testing_labels = np.asarray(testing_labels)
        
    def get_training_data_distribution(self):
        labels = np.unique(self.training_labels)
        dist = {}
        for i in labels:
            dist[i] = np.where(self.training_labels == i)
        return dist
    
    def get_testing_data_distribution(self):
        labels = np.unique(self.testing_labels)
        dist = {}
        for i in labels:
            dist[i] = np.where(self.testing_labels == i)
        return dist

###### Utility methods

In [145]:
def set_client_weights(weights, clients):
    for c in clients:
        c.model.set_weights(weights)

In [147]:
def split(array, partitions):
    k, m = divmod(len(array), partitions)
    return (array[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(partitions))

In [148]:
def mix_data(train_list, test_list):
    train_img, train_lbl, test_img, test_lbl = [[[] for i in range(10)] for i in range(4)]
    train_data = []
    for i in range(len(train_list[0])):
        train_data.append((train_list[0][i], train_list[1][i]))
    
    random.shuffle(train_data)
    print(len(train_data), train_data[0])
mix_data((train_images, train_labels), (test_images, test_labels))

60000 (array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
   

### Initialize variables

##### Objects

In [149]:
server = Server(mnist_model)
init_weights = np.asarray(server.model.get_weights())

# Populate the server's client list with clients holding data
server.clients = [Client(Dataset(train_data[i], train_data_labels[i], test_data[i], test_data_labels[i])) for i in range(NUMBER_OF_CLIENTS)]

# Execution

##### No client aggregation

In [150]:
# Initialize the clients' weights
set_client_weights(init_weights, server.clients)

# Perform federated averaging on the clients
server.federated_averaging(ROUNDS)

Epoch 1/3
186/186 [==============================] - 1s 3ms/step - loss: 1.9901 - sparse_categorical_accuracy: 0.9946 - val_loss: 67436.7969 - val_sparse_categorical_accuracy: 0.0850
Epoch 2/3
186/186 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 67436.7969 - val_sparse_categorical_accuracy: 0.0850
Epoch 3/3
186/186 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 67436.7969 - val_sparse_categorical_accuracy: 0.0850
Epoch 1/3
211/211 [==============================] - 1s 2ms/step - loss: 0.9379 - sparse_categorical_accuracy: 0.9953 - val_loss: 24262.4336 - val_sparse_categorical_accuracy: 0.1080
Epoch 2/3
211/211 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 1.0000 - val_loss: 24262.4336 - val_sparse_categorical_accuracy: 0.1080
Epoch 3/3
211/211 [==============================] - 0s 2ms/step - loss: 

##### Weighted averaging

##### Ensemble

##### Tuning

In [ ]:
https://www.tensorflow.org/tutorials/images/transfer_learning#fine_tuning
    


# Testing - delete later

In [158]:
# Evaluate clients
results = []
for c in server.clients:
    results.append(c.model.evaluate(c.dataset.testing_data, c.dataset.testing_labels, 128))
results

8/8 [==============================] - 0s 2ms/step - loss: 103.9552 - sparse_categorical_accuracy: 0.0970


[[65.9813461303711, 0.09799999743700027],
 [83.64959716796875, 0.07500000298023224],
 [59.338199615478516, 0.07800000160932541],
 [59.299659729003906, 0.16300000250339508],
 [73.35354614257812, 0.05400000140070915],
 [93.49577331542969, 0.09700000286102295],
 [74.37986755371094, 0.054999999701976776],
 [92.70693969726562, 0.08500000089406967],
 [85.04742431640625, 0.07800000160932541],
 [103.95523071289062, 0.09700000286102295]]

In [156]:
plt.plot(server.model.history['sparse_categorical_accuracy'], label='accuracy')
plt.plot(server.model.history['val_sparse_categorical_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(np.asarray(orderedListDataTest[0]),  np.asarray(orderedListLabelsTest[0]), verbose=2)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
NUMBER_OF_MODELS = 10
models = []
for i in range(NUMBER_OF_MODELS):
    
    models.append(
        Client(model.fit(
            np.asarray(orderedListData[i]), 
            np.asarray(orderedListLabels[i]), 
            epochs=10, 
            validation_data=(
                np.asarray(orderedListDataTest[i]), np.asarray(orderedListLabelsTest[i])
            )), len(orderedListData[i])))

In [ ]:
(models[0].model.get_weights() == models[1].model.get_weights()).all()

In [ ]:
np.array_equal(models[0].model.get_weights(), models[1].model.get_weights())

In [ ]:
models[0].sample_amount

In [ ]:
train_data = [[] for i in range(10)]
train_data

In [ ]:
[train_data[train_labels[i]].append(train_images[i]) for i in range(len(train_images))]

In [ ]:
len(train_data[9])

In [ ]:
train_labels[0]

In [42]:
test_weights = np.asarray(init_weights) + 1

In [45]:
(test_weights + np.asarray(init_weights)) / 2

a:\projects\python sesong 2\env\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


array([array([[0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       ...,
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5],
       [0.5, 0.5, 0.5, ..., 0.5, 0.5, 0.5]], dtype=float32),
       array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5

In [34]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])

model.compile(optimizer=tf.keras.optimizers.SGD(0.01),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )

In [35]:
np.asarray(model.get_weights())

array([array([[ 0.02979872, -0.07496471,  0.06127212, ...,  0.00026497,
         0.01988209,  0.02875005],
       [-0.06187219, -0.07302045, -0.06389229, ..., -0.04385625,
        -0.00143486,  0.0573824 ],
       [-0.06367228, -0.03734235, -0.01923789, ..., -0.04568401,
        -0.07224934, -0.05466868],
       ...,
       [ 0.06434707, -0.04622708, -0.01274758, ...,  0.0193691 ,
         0.08004429,  0.04332334],
       [ 0.0633579 ,  0.03055925, -0.06869059, ..., -0.06149226,
         0.0326076 ,  0.06383979],
       [-0.037656  , -0.00455572, -0.0554471 , ...,  0.03669924,
        -0.02400378, -0.00825565]], dtype=float32),
       array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [29]:
test_data[0][0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  11,
        150, 253, 202,  31,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  37,
        251, 251, 253, 107,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  